In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense

from keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 40
dataset = tf.keras.preprocessing.image_dataset_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/app2/Rice_Images_Data',
        shuffle=True,
        image_size = (IMAGE_SIZE, IMAGE_SIZE),
        batch_size = BATCH_SIZE
)

Found 440 files belonging to 4 classes.


In [4]:
class_names = dataset.class_names

In [5]:
def get_dataset_partition_tf (ds, train_split=0.7, val_split=0.2, test_split=0.1, shuffle=True, shuffle_size=10000):
    
    ds_size = len(ds) #dataset size
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
        
    train_size = int(train_split * ds_size) #train size converted to intger to avoid float
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_dataset_partition_tf(dataset)

In [7]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE) #buffer_size=tf.data.AUTOTUNE is to allow tf determine the batch size
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.3),
])

In [9]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE), #resize new inputs 
    layers.experimental.preprocessing.Rescaling(1.0/255) #scale down RGB
])

In [10]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 4
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, (3,3), activation='relu', input_shape = input_shape), #google Conv2D for all arguement. 32=no. of layers
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.1),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.1),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(64, activation='relu'), #dense layer of 64 neurons
    layers.Dense(n_classes, activation='softmax'), #softmax normalizes the prob of classes.  
])
model.build(input_shape=input_shape) 

In [11]:
model.compile(
    optimizer='Adagrad',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'] #track training process
)

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                      

In [13]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/40


9/9 [==============================] - 126s 344ms/step - loss: 1.4242 - accuracy: 0.2536 - val_loss: 1.3971 - val_accuracy: 0.2500
Epoch 2/40
9/9 [==============================] - 1s 162ms/step - loss: 1.4027 - accuracy: 0.2786 - val_loss: 1.3940 - val_accuracy: 0.2500
Epoch 3/40
9/9 [==============================] - 2s 173ms/step - loss: 1.3894 - accuracy: 0.2821 - val_loss: 1.3926 - val_accuracy: 0.2188
Epoch 4/40
9/9 [==============================] - 2s 214ms/step - loss: 1.3909 - accuracy: 0.2571 - val_loss: 1.3873 - val_accuracy: 0.2500
Epoch 5/40
9/9 [==============================] - 1s 163ms/step - loss: 1.3905 - accuracy: 0.2714 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 6/40
9/9 [==============================] - 1s 160ms/step - loss: 1.3876 - accuracy: 0.2714 - val_loss: 1.3885 - val_accuracy: 0.2500
Epoch 7/40
9/9 [==============================] - 1s 161ms/step - loss: 1.3890 - accuracy: 0.2429 - val_loss: 1.3901 - val_accuracy: 0.2500
Epoch 8/40
9/9 [=============

In [19]:
model.save('/content/drive/MyDrive/Colab Notebooks/app2/model.h5')